`yield from`
===

In [51]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        for x in gen:
            yield x
        yield after

In [16]:
g2 = lambda: (x**2 for x in range(3))
g3 = lambda: (x**3 for x in range(3))
g4 = lambda: (x**4 for x in range(3))

In [56]:
list(braced_chain(g2(), g3(), g4(), before='<', after='>'))

['<', 0, 1, 4, '>', '<', 0, 1, 8, '>', '<', 0, 1, 16, '>']

In [55]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        yield from gen
        yield after

Корутины
===

In [68]:
class Mean:
    def __init__(self):
        self._sum = 0
        self._n = 0
        
    def add(self, x):
        self._sum += x
        self._n += 1
        
        return x >= self._sum / self._n

In [69]:
m = Mean()

In [70]:
m.add(1)

True

In [71]:
m.add(10)

True

In [72]:
m.add(2)

False

In [73]:
m.add(3)

False

In [74]:
m.add(4)

True

In [75]:
def mean():
    result = None
    s = 0
    n = 0
    while True:
        x = yield result
        s += x
        n += 1
        result = x >= s / n

In [85]:
m = mean()

In [86]:
m

<generator object mean at 0x112652ca8>

In [87]:
next(m)

In [88]:
m.send(1)

True

In [89]:
m.send(10)

True

In [90]:
m.send(2)

False

In [91]:
m.send(3)

False

In [92]:
m.send(4)

True

In [302]:
def mean():
    result = None
    s = 0
    n = 0
    while True:
        x = yield result
        if x is None:
            break
        s += x
        n += 1
        result = x >= s / n

In [303]:
m = mean()

In [304]:
next(m)

In [305]:
m.send(1)

True

In [306]:
m.send(2)

True

In [307]:
m.send(None)

StopIteration: 

_Пример: докачка_

`yield from` для корутин
===

In [329]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        for x in gen:
            yield x
        yield after

In [330]:
b = braced_chain(g2(), mean())

In [331]:
list(b)

['{', 0, 1, 4, '}', '{', None, '}']

In [332]:
m = mean()
next(m)

In [333]:
b = braced_chain(g2(), m)

In [334]:
list(b)

['{', 0, 1, 4, '}', '{', '}']

In [335]:
m = mean()
next(m)
b = braced_chain(g2(), m)
next(b), next(b), next(b), next(b), next(b), next(b), b.send(1)

('{', 0, 1, 4, '}', '{', '}')

In [336]:
b.send(1)

StopIteration: 

In [338]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        yield from gen
        yield after

In [342]:
m = mean()
b = braced_chain(g2(), m)
(
    next(b), next(b), next(b), next(b), next(b),
    next(b), b.send(1), b.send(2), b.send(None)
)

('{', 0, 1, 4, '}', '{', None, True, '}')

In [343]:
b.send(13)

StopIteration: 

`return` в генераторе
===

In [11]:
def mean():
    result = None
    s = 0
    n = 0
    while True:
        x = yield result
        if x is None:
            break
        s += x
        n += 1
        result = x >= s / n
    return s / n

In [13]:
m = mean()
next(m), m.send(1), m.send(2), next(m)

StopIteration: 1.5

In [21]:
def braced_chain(*gens, before='{', after='}'):
    results = []
    for gen in gens:
        yield before
        result = yield from gen
        results.append(result)
        yield after
    return results

In [22]:
m = mean()
b = braced_chain(g2(), m)
(
    next(b), next(b), next(b), next(b), next(b),
    next(b), b.send(1), b.send(2), b.send(None)
)

('{', 0, 1, 4, '}', '{', None, True, '}')

In [23]:
next(b)

StopIteration: [None, 2.0]

`RestoringGet`
===

In [ ]:
logger = logging.getLogger(name)


class RestoringGet:
    MAX_RESTORES = 10

    def init(self, url, get_kwargs=None):
        if get_kwargs is None:
            get_kwargs = {}

        self._url = url
        self._kwargs = get_kwargs

    def get_generator(self):
        restores = 0
        offset = 0
        headers = {}
        while True:
            restores += 1
            if restores > self.MAX_RESTORES:
                raise TooManyRestores()

            response = requests.get(self._url, headers=headers, **self._kwargs)
            response.raise_for_status()

            real_length = yield response

            content_length = parse_int(response.headers.get('Content-Length'), None)
            if content_length is None or content_length + offset <= real_length:
                break

            logger.info('GET looks to be interrupted, trying to continue')
            offset = real_length
            headers = {'Range': 'bytes={}-'.format(offset)}

In [ ]:
gen = RestoringGet(...).get_generator()
response = next(get)
while response:
    file = save_response(response)
    try:
        response = get.send(file.size)
    except StopIteration:
        response = None

Communicate
===

In [2]:
import subprocess

In [25]:
p = subprocess.run(['python', '-c', 'open("/tmp/1133", "w").close()'])

In [26]:
p = subprocess.run(
    ['perl', '-E', 'say "out"; warn "err\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

In [27]:
p.stdout, p.stderr

(b'out\n', b'err\n')

In [3]:
from subprocess import Popen

In [32]:
p = Popen(
    ['perl', '-E', 'say "out"; warn "err\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

In [30]:
p.communicate()

(b'out\n', b'err\n')

In [39]:
while True:
    out = p.stdout.readline()
    err = p.stderr.readline()
    if out:
        print('out: {}'.format(out))
    if err:
        print('err: {}'.format(err))
    if not out and not err:
        break       

KeyboardInterrupt: 

In [38]:
p = Popen(
    ['perl', '-E', 'warn "err\n" x 1000000; say "out";'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

Threads
===

In [16]:
import threading
threads_num = 4

p = Popen(
    ['perl', '-E', 'warn "err\n"; say "out";'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

def log(fh, prefix):
    while True:
        line = fh.readline()
        if line:
            print('{}: {}'.format(prefix, line))
        else
            break

threads = [
    threading.Thread(target=log, args=(p.stdout, 'out')),
    threading.Thread(target=log, args=(p.stderr, 'err')),
]
for t in threads:
    t.start()
for t in threads:
    t.join()

err: b'err\n'out: b'out\n'



Selector
===

In [4]:
import selectors

sel = selectors.DefaultSelector()

In [5]:
p = Popen(
    ['perl', '-E', 'warn "err\n"; say "out"; warn "2\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
sel.register(p.stdout, selectors.EVENT_READ, lambda l: print('out: {}'.format(l)))
sel.register(p.stderr, selectors.EVENT_READ, lambda l: print('out: {}'.format(l)))

SelectorKey(fileobj=<_io.BufferedReader name=60>, fd=60, events=1, data=<function <lambda> at 0x10e07fb70>)

In [6]:
while True:
    events = sel.select()
    if not events:
        break
    for key, mask in events:
        callback = key.data
        line = key.fileobj.readline()
        if line:
            callback(line)
        else:
            sel.unregister(key.fileobj)

out: b'err\n'
out: b'out\n'
out: b'2\n'


`asyncio`
===

In [127]:
import asyncio
loop = asyncio.get_event_loop()

In [128]:
p = asyncio.create_subprocess_exec(
    'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
    stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
)

In [129]:
p

<generator object create_subprocess_exec at 0x10e30f0f8>

In [130]:
next(p)

<Future pending>

In [132]:
@asyncio.coroutine
def execute():
    p = yield from asyncio.create_subprocess_exec(
        'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    print(p)

In [133]:
execute()

<generator object execute at 0x10e30fa40>

In [134]:
loop.run_until_complete(execute())

<Process 16994>


In [135]:
@asyncio.coroutine
def log(fh, prefix):
    while True:
        line = yield from fh.readline()  ###
        if line:
            print('{}: {}'.format(prefix, line))
        else:
            break

In [136]:
@asyncio.coroutine
def execute(loop):
    p = yield from asyncio.create_subprocess_exec(
        'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    
    loop.create_task(log(p.stdout, 'stdout'))
    loop.create_task(log(p.stderr, 'stderr'))
    
    yield from p.wait()

In [137]:
loop.run_until_complete(execute(loop))

stderr: b'err\n'
stderr: b'2\n'
stdout: b'out\n'


`async` / `await`
===

In [138]:
async def log(fh, prefix):
    while True:
        line = await fh.readline()
        if line:
            print('{}: {}'.format(prefix, line))
        else:
            break

In [139]:
async def execute(loop):
    p = await asyncio.create_subprocess_exec(
        'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    
    loop.create_task(log(p.stdout, 'stdout'))
    loop.create_task(log(p.stderr, 'stderr'))
    
    await p.wait()

In [140]:
loop.run_until_complete(execute(loop))

stderr: b'err\n'
stderr: b'2\n'
stdout: b'out\n'


`aiohttp`
===

In [56]:
import aiohttp
import asyncio

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            return await response.text()  # return yield from
        else:
            return f'ERROR: {response.status}'

async def download_wiki(article):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://de.wikipedia.org/{}'.format(article))
        return html[:15]

loop = asyncio.get_event_loop()
tasks = asyncio.gather(
    download_wiki('wiki/Évariste_Galois'),
    download_wiki('wiki/Alan_Turing'),
    download_wiki('zzz'),
)

In [57]:
tasks

<_GatheringFuture pending>

In [58]:
loop.run_until_complete(tasks)

['<!DOCTYPE html>', '<!DOCTYPE html>', 'ERROR: 404']

Сервер
===

In [5]:
from aiohttp import web